In [1]:
import pandas as pd
import numpy as np

In [2]:
#read data
calories1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/calories.json')
distance1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/distance.json')
l_act1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/lightly_active_minutes.json')
m_act1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/moderately_active_minutes.json')
r_heart1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/resting_heart_rate.json')
sedentary1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/sedentary_minutes.json')
sleep_score1 = pd.read_csv('../osfstorage-archive/pmdata/p01/fitbit/sleep_score.csv')
sleep1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/sleep.json')
steps1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/steps.json')
hr_zone1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/time_in_heart_rate_zones.json')
v_act1 = pd.read_json('../osfstorage-archive/pmdata/p01/fitbit/very_active_minutes.json')

In [3]:
#data frequency to date using sum
calories_processed = calories1.resample('D', on = 'dateTime').value.sum() 
distance_processed = distance1.resample('D', on = 'dateTime').value.sum()
steps_processed = steps1.resample('D', on = 'dateTime').value.sum()

In [4]:
#resting heart rate to simple df
r_heart1['value']=r_heart1['value'].apply(lambda x: x['value'])
r_heart_processed=r_heart1

In [5]:
#delete time in sleep score time stamp
sleep_score1['timestamp']=sleep_score1['timestamp'].apply(lambda x: x[:10])
#drop unnecessary column
sleep_score_processed = sleep_score1.drop(columns='sleep_log_entry_id')

In [6]:
#dictionary with heart rate zone minutes to df
hr_zone_value = pd.DataFrame([eval(str(a).replace('{\'valuesInZones\': ','').replace('}}','}')) for a in hr_zone1['value']])
hr_zone_processed = pd.concat([hr_zone1['dateTime'], hr_zone_value], axis=1)

In [7]:
#unify index
l_act_processed = l_act1.set_index('dateTime')
m_act_processed = m_act1.set_index('dateTime')
r_heart_processed = r_heart_processed.set_index('dateTime')
sedentary_processed = sedentary1.set_index('dateTime')
sleep_score_processed = sleep_score_processed.set_index('timestamp')
sleep_processed = sleep1.set_index('dateOfSleep')
hr_zone_processed = hr_zone_processed.set_index('dateTime')
v_act_processed = v_act1.set_index('dateTime')
#index to DateTimeIndex
sleep_score_processed.index = pd.to_datetime(sleep_score_processed.index)

In [8]:
#get dummy values for mainsleep true(1)/false(0)
sleep_processed['mainSleep'] = pd.get_dummies(sleep_processed['mainSleep'])[True].fillna(0)
#drop unnecessary columns
sleep_processed = sleep_processed.drop(['logId','startTime','endTime','type','infoCode','levels'], axis=1)
#index to DateTimeIndex
sleep_processed.index = pd.to_datetime(sleep_processed.index)

In [9]:
#read prediction target
wellness = pd.read_csv('../osfstorage-archive/pmdata/p01/pmsys/wellness.csv') 
wellness['effective_time_frame'] = wellness['effective_time_frame'].apply(lambda x: x[:10])
wellness = wellness.rename(columns={"effective_time_frame": "dateTime"})
readiness = wellness.loc[:,['dateTime','readiness']].set_index('dateTime')
readiness.index = pd.to_datetime(readiness.index)

In [10]:
#drop duplicated records
readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed = [x[~x.index.duplicated(keep='first')] for x in [readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed]]

In [11]:
#concat data
P01 = pd.concat([readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed], axis=1)

In [12]:
#rename columns
P01.columns=['readiness','calories','distance','l_act','m_act','resting_heart','sedentary','sleep_overall_score','sleep_composition_score',
             'sleep_revitalization_score', 'sleep_duration_score',
             'deep_sleep_in_minutes', 'sleep_resting_heart_rate','sleep_restlessness',
             'sleep_duration','sleep_minutesToFallAsleep','sleep_minutesAsleep','sleep_minutesAwake',
             'sleep_minutesAfterWakeup','sleep_timeInBed','sleep_efficiency',
             'main_sleep','steps',
             'hr_below_zone1', 'hr_zone1', 'hr_zone3','hr_zone2','v_act']

In [13]:
P01

,readiness,calories,distance,l_act,m_act,resting_heart,sedentary,sleep_overall_score,sleep_composition_score,sleep_revitalization_score,...,sleep_minutesAfterWakeup,sleep_timeInBed,sleep_efficiency,main_sleep,steps,hr_below_zone1,hr_zone1,hr_zone3,hr_zone2,v_act
2019-11-01,5.0,4009.10,1442400,245,58,53.741074,636,76.0,18.0,20.0,...,NaN,NaN,NaN,NaN,17873,1254.0,164.0,0.0,3.0,72
2019-11-02,6.0,3533.56,1058480,202,43,52.881497,709,79.0,20.0,22.0,...,0.0,430.0,97.0,1.0,13118,1262.0,137.0,0.0,0.0,56
2019-11-03,8.0,3748.73,1146085,262,63,53.222024,648,71.0,16.0,17.0,...,0.0,422.0,96.0,1.0,14312,1218.0,141.0,0.0,0.0,45
2019-11-04,8.0,3353.38,885970,230,23,54.311141,760,68.0,15.0,15.0,...,0.0,399.0,96.0,1.0,10970,1315.0,100.0,0.0,0.0,28
2019-11-05,8.0,3794.63,1371470,263,22,52.259110,737,73.0,15.0,22.0,...,1.0,362.0,99.0,1.0,16186,1300.0,94.0,12.0,7.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,6.0,3383.74,922120,181,28,50.232508,821,72.0,19.0,20.0,...,0.0,345.0,97.0,1.0,11532,1347.0,82.0,0.0,3.0,65
2020-03-28,8.0,3967.80,1326460,207,69,51.125174,679,67.0,16.0,14.0,...,0.0,402.0,96.0,1.0,16448,1232.0,173.0,0.0,5.0,83
2020-03-29,8.0,2832.94,541076,145,19,53.070500,840,62.0,16.0,14.0,...,0.0,474.0,99.0,1.0,6698,1319.0,49.0,0.0,0.0,22
2020-03-30,8.0,2696.81,384244,161,6,52.175083,842,78.0,19.0,20.0,...,3.0,423.0,98.0,1.0,4761,1384.0,23.0,0.0,2.0,8
